<a href="https://colab.research.google.com/github/Prishletsova/Gonna_make_it_work/blob/main/%D0%9B%D0%B5%D0%BA%D1%86%D0%B8%D1%8F_21_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Декоратор

Функции в Python'e являются объектами, а значит:


*   могут быть связаны с переменной;
*   могут быть определены одна в другой.



# Декорирование методов

Функции и методы в Python'e - это практически одно и то же, за исключением того, что методы всегда ожидают первым параметром ссылку на сам объект (self).

Это значит, что мы можем создавать декораторы для методов так же, как и функций, просто не забывая про self.

In [ ]:
def method_friendly_decorator(method_to_decorate):
    def wrapper(self, lie):
        lie = lie - 3
        return method_to_decorate(self, lie)
    return wrapper

class Lucy(object):
    def __init__(self):
        self.age = 32
    @method_friendly_decorator
    def say_your_age(self, lie):
        print('Мне %s, а ты бы сколько дал?' % (self.age + lie))

l = Lucy()
l.say_your_age(-3)


Мне 26, а ты бы сколько дал?


Если создавать общий декоратор, чтобы его можно было применить к любой функции или методу, то стоит воспользоваться тем, что:


*   *args распаковывает список args
*   **kwargs распаковывает словарь kwargs



# Декораторы классов

**Согласно модели данных Python, язык предлагает три вида методов:**

Метод экземпляра класса

Метод класса

Статический метод

In [ ]:
class ToyClass:
    def instancemethod(self):
        return 'instance method called', self

    @classmethod
    def classmethod(cls):
        return 'class method called', cls

    @staticmethod
    def staticmethod():
        return 'static method called'

**Методы экземпляра класса**



*   Методы экземпляра класса принимают объект класса как первый аргумент, который принято называть self и который указывает на сам экземпляр. Количество параметров метода не ограничено.  
*   Используя параметр self, мы можем менять состояние объекта и обращаться к другим его методам и параметрам. К тому же, используя атрибут self.class, мы получаем доступ к атрибутам класса и возможности менять состояние самого класса. То есть методы экземпляров класса позволяют менять как состояние определенного объекта, так и класса.



**Статистические методы**

Статистические методы декларируются при помощи декоратора staticmethod. 
Им не нужен определенный первый аргумент (ни self, ни cls).

Их можно воспринимать как методы, которые "не знают, к какому методу относятся". Таким образом, статистические методы прикреплены к классу лишь для удобства и не могут менять

Когда и какой метод стоит использоватьь

In [ ]:
from datetime import date

class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @classmethod
    def from_birth_year(cls, name, year):
        return cls(name, date.today().year - year)

    @staticmethod
    def is_adult(age):
        return age > 18

In [ ]:
person1 = Person('Sarah', 25)
person2 = Person.from_birth_year('Roark', 1994)

In [ ]:
person1.name, person1.age

('Sarah', 25)

In [ ]:
person2.name, person2.age

('Roark', 29)

In [ ]:
Person.is_adult(25)

True

# Пример 1

In [ ]:
from math import pi
class Cylinder:
    @staticmethod
    def make_area(d, h):
        circle = pi *d ** 2 / 4
        side = pi * d * h
        return round(circle*2 + side, 2)
    def __init__(self, diameter, high):
        self.dia = diameter
        self.h = high
        self.area = self.make_area(diameter, high)

a = Cylinder(1, 2)
print(a.area)
print(a.make_area(2, 2))

7.85
18.85


# Задача 1

In [ ]:
from abc import ABC, abstractmethod
class Hero:
    def __init__(self):
        self.positive_effects = []
        self.negative_effects = []
        self.stats = {
            "HP": 128,
            "MP": 42, 
            "SP": 100,
            "Strength": 15, 
            "Perception": 4,
            "Endurance": 8,
            "Charisma": 2,
            "Intelligence": 3, 
            "Agility": 8,
            "Luck": 1
        }
        
    def get_positive_effects(self):
        return self.positive_effects.copy()

    def get_negative_effects(self):
        return self.negative_effects.copy()

    def get_stats(self):
        return self.stats.copy()

In [ ]:
class AbstractEffect(Hero, ABC):
    def __init__(self, base):
        self.base = base

    @abstractmethod
    def get_positive_effects(self):
        return self.positive_effects

    @abstractmethod
    def get_negative_effects(self):
        return self.negative_effects

    @abstractmethod
    def get_stats(self):
        pass


class AbstractPositive(AbstractEffect):
    def get_negative_effects(self):
        return self.base.get_negative_effects()


class AbstractNegative(AbstractEffect):
    def get_positive_effects(self):
        return self.base.get_positive_effects()


class Berserk(AbstractPositive):
    def get_stats(self):
        stats = self.base.get_stats()
        stats["HP"] += 50
        stats["Strength"] += 7
        stats["Endurance"] += 7
        stats["Agility"] += 7
        stats["Luck"] += 7
        stats["Perception"] -= 3
        stats["Charisma"] -= 3
        stats["Intelligence"] -= 3
        return stats

    def get_positive_effects(self):
        return self.base.get_positive_effects() + ["Berserk"]